In [2]:
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np
import statistics as stat
from scipy import stats
lone_pine = pd.read_csv('lone_pine.csv')
crsp_data = pd.read_csv('StockPriceMonthly.csv', encoding='utf8', low_memory=False)

# pre-processing

In [3]:
lone_pine.drop(['% Port', '% OS', 'Hist', 'Change', 'Change.1'], axis=1, inplace=True)

In [4]:
lone_pine['Ticker'] = lone_pine['Ticker'].astype('str')
lone_pine['Class'] = lone_pine['Class'].astype('str')
def ab(aa):
    if '.' in aa:
        return aa.split('.')[0]
    if '/' in aa:
        return aa.split('/')[0]
    else:
        return aa
lone_pine['Ticker'] = lone_pine['Ticker'].apply(ab)

In [5]:
def abc(aa):
    aa = aa.split('/')
    year = aa[2]
    month = aa[0]
    day = aa[1]
    return year+month
lone_pine['Date'] = lone_pine['Date'].apply(abc)

In [6]:
lone_pine = lone_pine[lone_pine['Class']!='PUT'].loc[lone_pine['Class']!='CALL']
lone_pine = lone_pine[lone_pine['Class']!='put'].loc[lone_pine['Class']!='call']
lone_pine = lone_pine[lone_pine['Class']!='Put'].loc[lone_pine['Class']!='Call']

In [7]:
def class_3str(df_col):
    df_col = df_col.replace(' ', '')
    df_col = df_col[0:3]
    df_col = df_col.upper()
    return df_col
lone_pine['Class'] = lone_pine['Class'].apply(class_3str)

In [8]:
lone_pine_not = lone_pine[lone_pine['Ticker']!='nan']
lone_pine_is = lone_pine[lone_pine['Ticker']=='nan']

In [9]:
lone_pine_not.drop('Name', axis=1, inplace=True)
lone_pine_not = lone_pine_not.groupby(['Ticker','Class','Date'])[['Shares', 'Value']].agg('sum').reset_index().copy()

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [10]:
lone_pine_is.loc[:, 'Ticker'] = lone_pine_is['Name']
lone_pine_is.drop('Name', axis=1, inplace=True)
lone_pine_is = lone_pine_is.groupby(['Ticker','Class','Date'])[['Shares', 'Value']].agg('sum').reset_index().copy()

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
date = sorted(list(set(lone_pine_not['Date'])))

In [12]:
new_lone_pine = pd.DataFrame()

In [13]:
for num in range(len(date)-1):
    first_date = date[num]
    second_date = date[num+1]
    first_df = lone_pine_not[lone_pine_not['Date']==first_date]
    second_df = lone_pine_not[lone_pine_not['Date']==second_date]
    first_df = first_df.drop(['Date'],axis=1)
    first_df.columns = ['Ticker', 'Class', 'Last_Shares', 'Last_Value']
    merge_df = second_df.merge(first_df, on=['Ticker','Class'], how='left')
    if len(merge_df) != len(second_df):
        print(second_date)
    new_lone_pine = pd.concat([new_lone_pine, merge_df], ignore_index=True)

In [14]:
new_lone_pine_is = pd.DataFrame()

In [15]:
for num in range(len(date)-1):
    first_date = date[num]
    second_date = date[num+1]
    first_df = lone_pine_is[lone_pine_is['Date']==first_date]
    second_df = lone_pine_is[lone_pine_is['Date']==second_date]
    first_df = first_df.drop(['Date'],axis=1)
    first_df.columns = ['Ticker', 'Class', 'Last_Shares', 'Last_Value']
    merge_df = second_df.merge(first_df, on=['Ticker','Class'], how='left')
    if len(merge_df) != len(second_df):
        print(second_date)
    new_lone_pine_is = pd.concat([new_lone_pine_is, merge_df], ignore_index=True, sort=False)

In [16]:
new_lone_pine = pd.concat([new_lone_pine, lone_pine_not[lone_pine_not['Date']==date[0]]], ignore_index=True, sort=False)
new_lone_pine_is = pd.concat([new_lone_pine_is, lone_pine_is[lone_pine_is['Date']==date[0]]],ignore_index=True, sort=False)
lone_pine_a = pd.concat([new_lone_pine, new_lone_pine_is], ignore_index=True, sort=False)

In [17]:
lone_pine_a = lone_pine_a.replace(np.nan, 0)

In [18]:
lone_pine_a.loc[:, 'Shares_Change'] = lone_pine_a['Shares'] - lone_pine_a['Last_Shares']
lone_pine_a.loc[:, 'Value_Change'] = lone_pine_a['Value'] - lone_pine_a['Last_Value']

In [19]:
lone_pine_a.drop(['Last_Shares', 'Last_Value'], axis=1, inplace=True)

In [20]:
add_df = pd.DataFrame(columns=lone_pine_a.columns)
for index in tqdm_notebook(range(len(lone_pine_a))):
    each_data = lone_pine_a.loc[index]
    each_data = list(each_data)
    date = int(each_data[2])
    if date % 4 == 0:
        date = date - 12 + 100
    each_data[2] = str(date+1)
    each_data[5] = 0
    each_data[6] = 0
    add_df.loc[-1] = each_data
    add_df = add_df.reset_index(drop=True)
    date = int(each_data[2])
    each_data[2] = str(date+1)
    each_data[5] = 0
    each_data[6] = 0
    add_df.loc[-1] = each_data
    add_df = add_df.reset_index(drop=True)
lone_pine_a = pd.concat([lone_pine_a, add_df], ignore_index=True, sort=False)

In [21]:
lone_pine_a.drop(lone_pine_a[lone_pine_a['Shares']==0].loc[lone_pine_a['Value']==0].loc[lone_pine_a['Shares_Change']==0].loc[lone_pine_a['Value_Change']==0].index, inplace=True)

In [22]:
def class_(df_col):
    if df_col == 'CLA':
        return 'A'
    elif df_col == 'CLB':
        return 'B'
    elif df_col == 'CLC':
        return 'C'
    elif df_col == 'CLD':
        return 'D'
    else:
        return df_col
lone_pine_a['Class'] = lone_pine_a['Class'].apply(class_)

In [23]:
def division(df):
    if df['Shares_Change'] == 0 and df['Shares'] == 0:
        return 0
    elif df['Shares'] == 0 and df['Value'] == 0:
        return df['Value_Change'] / df['Shares_Change']
    else:
        return df['Value'] / df['Shares']
lone_pine_a['price'] = lone_pine_a.apply(division, axis=1)

In [24]:
def abcd(aa):
    return str(aa)[0:6]
crsp_data['date'] = crsp_data['date'].apply(abcd)

In [25]:
crsp_data.columns = ['PERMNO', 'Date', 'Ticker', 'COMNAM', 'Class', 'DIVAMT', 'PRC', 'VOL']

In [26]:
def to_abs(df_col):
    return abs(df_col)
crsp_data_c = crsp_data[['Date', 'Ticker', 'PRC', 'Class']]
crsp_data_c = crsp_data_c[crsp_data_c['Ticker'].isin(list(set(lone_pine_a['Ticker'])))]
crsp_data_c = crsp_data_c[crsp_data_c['PRC'].notnull()].drop_duplicates(keep='first')
crsp_data_c['PRC'] = crsp_data_c['PRC'].apply(to_abs)

In [27]:
# crsp_data_c = crsp_data_c.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)
# df_list = []
# for each in tqdm_notebook(crsp_data_c.groupby(by='Ticker')):
#     ticker, df = each
#     df.loc[:, 'Last_PRC'] = df['PRC'].shift()
#     df_list.append(df)

In [28]:
import multiprocessing as mp
import time
from function import test_function
if __name__ == '__main__':
    a = time.time()
    pool_list = [i for i in crsp_data_c.groupby(by='Ticker')]
    pool = mp.Pool(processes=5)
    df_list = pool.map(test_function.last_price, pool_list)
    b = time.time()
    print(b - a)

2.5651121139526367


In [29]:
crsp_data_c = pd.concat(df_list)

In [30]:
lone_pine_all = lone_pine_a.merge(crsp_data_c, on=['Date', 'Ticker'], how = 'left')

In [31]:
for ticker in list(set(list(lone_pine_all['Ticker']))):
    if len(lone_pine_a[lone_pine_a['Ticker']==ticker].reset_index(drop=True)) != len(lone_pine_all[lone_pine_all['Ticker']==ticker].reset_index(drop=True)):
        drop_df = lone_pine_all[lone_pine_all['Ticker']==ticker].loc[lone_pine_all['Class_x']!=lone_pine_all['Class_y']]
        lone_pine_all.drop(list(drop_df.index), inplace=True)

In [34]:
error = lone_pine_all[lone_pine_all['PRC'].isnull()]

In [35]:
test = lone_pine_all
new_error = []
for index in tqdm_notebook(error.index):
    each = error.loc[index]
    df = crsp_data[crsp_data['Ticker']==each['Ticker']]
    try:
        if int(each['Date']) > int(list(df['Date'])[-1]):
            test = test.drop(index)
        else:
            new_error.append(list(each))
    except:
        new_error.append(list(each))
lone_pine_all = test

In [36]:
new_error = pd.DataFrame(new_error)
new_error.columns = lone_pine_all.columns
new_error = new_error.sort_values(by='Date').reset_index(drop=True)

In [37]:
df_list = []
for each in tqdm_notebook(new_error.groupby(by='Ticker')):
    ticker, df = each
    df.loc[:, 'PRC'] = df['price']
    df.loc[:, 'Last_PRC'] = df['PRC'].shift()
    df_list.append(df)
new_error = pd.concat(df_list)

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [38]:
lone_pine_all = pd.concat([lone_pine_all[lone_pine_all['PRC'].notnull()], new_error], ignore_index=True, sort=False)

In [40]:
lone_pine_all = lone_pine_all.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)

# industry

In [144]:
industry = pd.read_csv('industry2.csv')
lone_pine = pd.read_csv('lone_pine.csv')

In [145]:
lone_pine = lone_pine[['Ticker', 'Class', 'Date']]
lone_pine['Ticker'] = lone_pine['Ticker'].astype('str')
lone_pine['Class'] = lone_pine['Class'].astype('str')
def ab(aa):
    if '.' in aa:
        return aa.split('.')[0]
    if '/' in aa:
        return aa.split('/')[0]
    else:
        return aa
lone_pine['Ticker'] = lone_pine['Ticker'].apply(ab)
def abc(aa):
    aa = aa.split('/')
    year = aa[2]
    month = aa[0]
    day = aa[1]
    return year+month
lone_pine['Date'] = lone_pine['Date'].apply(abc)
lone_pine = lone_pine[lone_pine['Class']!='PUT'].loc[lone_pine['Class']!='CALL']
lone_pine = lone_pine[lone_pine['Class']!='put'].loc[lone_pine['Class']!='call']
lone_pine = lone_pine[lone_pine['Class']!='Put'].loc[lone_pine['Class']!='Call']
def class_3str(df_col):
    df_col = df_col.replace(' ', '')
    df_col = df_col[0:3]
    df_col = df_col.upper()
    return df_col
lone_pine['Class'] = lone_pine['Class'].apply(class_3str)

In [147]:
lone_pine_indus = lone_pine.merge(industry[['tic', 'gsector', 'ggroup']], left_on='Ticker', right_on='tic', how='left')

In [234]:
def indus_sort(data, date):
    pre_date = []
    date_list = sorted(list(set(data['Date'])))
    last_date = int(date[-1])
    # insert newest date
    if last_date % 4 == 0:
        date_list.append(str(last_date+100-12+3))
    else:
        date_list.append(str(last_date+3))
    # get pre 4 seasons date
    try:
        index = date_list.index(date)
        for r in range(4):
            if index > 0:
                index-=1
                pre_date.append(date_list[index])
    except:
        return 'date error'
    
    data = data[data['Date'].isin(pre_date)]
    data2 = data[data['gsector'].notnull()]
    data4 = data[data['ggroup'].notnull()]
    
    dic2 = {}
    for each_gsector in list(data2['gsector']):
        if each_gsector not in dic2:
            dic2[each_gsector] = 0
        dic2[each_gsector]+=1
    dic2 = {k: v for k, v in sorted(dic2.items(), reverse=True, key=lambda item: item[1])}
    for num in dic2:
        dic2[num] = (dic2[num] / len(list(data2[data2['gsector'].notnull()]['gsector'])))*100
    dic2_new = {}
    percent = 0
    dic2_key = list(dic2.keys())
    for num in dic2_key:
        percent = percent + dic2[num]
        dic2_new[num] = dic2[num]
        if percent > 80:
            break
    if len(dic2_new) < 5:
        for n in list(dic2)[:5]:
            dic2_new[n] = dic2[n]
    
    dic4 = {}
    for each_ggroup in list(data4['ggroup']):
        if each_ggroup not in dic4:
            dic4[each_ggroup] = 0
        dic4[each_ggroup]+=1
    dic4 = {k: v for k, v in sorted(dic4.items(), reverse=True, key=lambda item: item[1])}
    for num in dic4:
        dic4[num] = (dic4[num] / len(list(data4[data4['ggroup'].notnull()]['ggroup'])))*100
    dic4_new = {}
    dic4_key = list(dic4.keys())
    for num in dic4_key:
        if dic4[num] > 5:
            dic4_new[num] = dic4[num]
        else:
            break
    
    return dic2_new, dic4_new

In [237]:
indus_sort(lone_pine_indus, '200806')

({45.0: 22.435897435897438,
  35.0: 16.666666666666664,
  15.0: 11.538461538461538,
  25.0: 11.538461538461538,
  50.0: 10.897435897435898,
  40.0: 10.897435897435898},
 {4510.0: 14.102564102564102,
  1510.0: 11.538461538461538,
  4020.0: 9.615384615384617,
  3520.0: 8.333333333333332,
  2550.0: 8.333333333333332,
  3510.0: 8.333333333333332,
  1010.0: 7.051282051282051,
  2010.0: 6.41025641025641,
  5010.0: 5.769230769230769,
  4520.0: 5.769230769230769,
  5020.0: 5.128205128205128})

# revenue

In [42]:
lone_pine_all.drop(['Class_x', 'price'], axis=1, inplace=True)
lone_pine_all.columns = ['Ticker', 'Date', 'Shares', 'Value', 'Shares_Change', 'Value_Change', 'Price',
                         'Class', 'Last_Price']

In [43]:
new_share = lone_pine_all.loc[lone_pine_all['Shares']==lone_pine_all['Shares_Change']]
new_share.loc[:,'New_Value'] = new_share['Shares'] * new_share['Price']

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [44]:
new_share.to_csv('new_share.csv', index=False)

In [45]:
sell_share = lone_pine_all[lone_pine_all['Shares']==0]
sell_share.loc[:,'New_Value'] = sell_share['Shares_Change'] * sell_share['Price']

In [46]:
sell_share.to_csv('sell_share.csv', index=False)

In [47]:
more_share = lone_pine_all[lone_pine_all['Shares_Change']>0].loc[lone_pine_all['Shares']!=lone_pine_all['Shares_Change']]
more_share.loc[:,'New_Value'] = more_share['Shares'] * more_share['Price']

In [48]:
more_share.to_csv('more_share.csv', index=False)

In [49]:
less_share = lone_pine_all[lone_pine_all['Shares_Change']<0].loc[lone_pine_all['Shares']!=0]
less_share.loc[:,'New_Value'] = (less_share['Shares'] + abs(less_share['Shares_Change'])) * less_share['Price']

In [50]:
less_share.to_csv('less_share.csv', index=False)

In [51]:
same_share = lone_pine_all[lone_pine_all['Shares_Change']==0]
same_share.loc[:,'New_Value'] = same_share['Shares'] * same_share['Price']

In [52]:
same_share.to_csv('same_share.csv', index=False)

In [53]:
lone_pine_all.loc[:,'New_Value'] = lone_pine_all['Shares'] * lone_pine_all['Price']

In [54]:
total_asset = []
for date in list(set(lone_pine_all['Date'])):
    lone_pine_all_t = lone_pine_all[lone_pine_all['Date']==date]
    total_asset.append(sum(lone_pine_all_t['New_Value']))

In [55]:
revenue = []
for date in list(set(lone_pine_all['Date'])):
    lone_pine_all_t = lone_pine_all[lone_pine_all['Date']==date]

    sell_shares = lone_pine_all_t[lone_pine_all_t['Shares']==0]
    more_shares = lone_pine_all_t[lone_pine_all_t['Shares_Change']>0].loc[lone_pine_all_t['Shares']!=lone_pine_all_t['Shares_Change']]
    less_shares = lone_pine_all_t[lone_pine_all_t['Shares_Change']<0].loc[lone_pine_all_t['Shares']!=0]
    same_shares = lone_pine_all_t[lone_pine_all_t['Shares_Change']==0]

    sell_shares.loc[:,'diff'] = abs(sell_shares['Shares_Change']) * ((sell_shares['Price'] - sell_shares['Last_Price']) / 2)
    more_shares.loc[:,'diff'] = (more_shares['Shares'] - more_shares['Shares_Change']) * (more_shares['Price'] - more_shares['Last_Price'])
    less_shares.loc[:,'diff'] = less_shares['Shares'] * (less_shares['Price'] - less_shares['Last_Price']) + abs(less_shares['Shares_Change']) * ((less_shares['Price'] - less_shares['Last_Price']) / 2)
    same_shares.loc[:,'diff'] = same_shares['Shares'] * (same_shares['Price'] - same_shares['Last_Price'])
    
    sell_shares = sell_shares[sell_shares['diff'].notnull()]
    more_shares = more_shares[more_shares['diff'].notnull()]
    less_shares = less_shares[less_shares['diff'].notnull()]
    same_shares = same_shares[same_shares['diff'].notnull()]
    
    revenue.append(sum(sell_shares['diff']) + sum(more_shares['diff']) + sum(less_shares['diff']) + sum(same_shares['diff']))

In [56]:
revenue_df = pd.DataFrame({'date':list(set(lone_pine_all['Date'])),
                          'asset':total_asset,
                          'revenue':revenue})

In [57]:
revenue_df = revenue_df.sort_values(by='date').reset_index(drop=True)

In [58]:
revenue_df['last_asset'] = revenue_df['asset'].shift()

In [59]:
revenue_df['return_rate'] = round(revenue_df['revenue'] / revenue_df['last_asset'] * 100, 4)

In [60]:
revenue_df = revenue_df[0:-2].reset_index(drop=True)

In [61]:
revenue_df

,date,asset,revenue,last_asset,return_rate
0,200412,5.495695e+09,0.000000e+00,NaN,NaN
1,200501,5.393992e+09,-1.017030e+08,5.495695e+09,-1.8506
2,200502,5.439646e+09,4.565398e+07,5.393992e+09,0.8464
3,200503,7.240458e+09,-1.092002e+08,5.439646e+09,-2.0075
4,200504,6.989394e+09,-2.510637e+08,7.240458e+09,-3.4675
5,200505,7.383804e+09,3.944094e+08,6.989394e+09,5.6430
6,200506,8.001011e+09,2.845453e+08,7.383804e+09,3.8536
7,200507,7.954283e+09,-4.672828e+07,8.001011e+09,-0.5840
8,200508,7.865848e+09,-8.843514e+07,7.954283e+09,-1.1118
9,200509,7.425471e+09,7.608325e+07,7.865848e+09,0.9673


In [409]:
revenue_df.to_csv('revenue.csv', index=False)

# LinearRegression

In [342]:
from sklearn.linear_model import LinearRegression

In [183]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# sc = StandardScaler()
# sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
# x_train_std = sc.transform(x_train)
# x_test_std = sc.transform(x_test)

In [185]:
lr = LinearRegression(alpha)
lr.fit(x_train_std, y_train['return_rate'].values)
y_predict_train = lr.predict(x_train_std)
score_train = r2_score(y_train['return_rate'].values, y_predict_train)
y_predict = lr.predict(x_test_std)
score=r2_score(y_test['return_rate'].values, y_predict)
e = mean_squared_error(y_test['return_rate'].values, y_predict)
coeff_used = np.sum(lr.coef_!=0)
print('p-value:', list(f_regression(x_test, y_predict)[1]))
print('coef:', lr.coef_)
print('score_train:',score_train)
print('score_prediction:',score)
print('e:',e)
# list(lr.coef_) + [score_train, score, e, coeff_used]

p-value: [0.5364073564694, 0.7714801695018272, 0.18793909123183322]
coef: [-0.57247408  0.23711452 -0.94635621  0.18319545 -0.31237053]
score_train: 0.03589049297583946
score_prediction: -0.03566527760886751
e: 27.781236821420062


In [467]:
fama_5 = fama_5.reset_index(drop=True)
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']].values
y = fama_5[['return_rate']].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# sc = StandardScaler()
# sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
# x_train_std = sc.transform(x_train)
# x_test_std = sc.transform(x_test)

In [468]:
lr = LinearRegression(fit_intercept=True)
lr.fit(x, y)
y_predict_train = lr.predict(x)
score_train = r2_score(y, y_predict_train)
# y_predict = lr.predict(x_test)
# score=r2_score(y_train['return_rate'].values, y_predict_train)
e = mean_squared_error(y, y_predict_train)
coeff_used = np.sum(lr.coef_!=0)
print('p-value:', list(f_regression(x, y)[1]))
print('coef:', lr.coef_)
print('score_train:',score_train)
print('score_prediction:',score)
print('e:',e)
print(lr.intercept_)
# list(lr.coef_) + [score, e, coeff_used]

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


p-value: [1.7077306071700508e-37, 8.60780085941154e-05, 6.892823204276131e-05, 1.9131572454502635e-05, 0.48342675604247676]
coef: [[ 1.00505773  0.04930608  0.13185248 -0.5469867  -0.3264541 ]]
score_train: 0.6838660785711304
score_prediction: 0.8572360985613784
e: 8.231465820219352
[0.22820167]


In [469]:
f_regression(x,y)

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(array([271.36490153,  16.1630756 ,  16.62366372,  19.31340754,
          0.49320973]),
 array([1.70773061e-37, 8.60780086e-05, 6.89282320e-05, 1.91315725e-05,
        4.83426756e-01]))

In [110]:
p1 = [93]
p2 = [85, 84, 73, 88, 80, 85, 77, 68, 95, 80, 92, 90, 78, 95, 94, 84, 72, 81]
p3 = [80, 73, 78, 78, 74, 79, 96, 74, 90, 90, 82, 88, 82, 90, 82, 92, 88, 96, 87, 91, 92, 86, 98]

In [111]:
def point_change(n):
    if 80 <= n <= 100:
        return 4
    elif 70 <= n < 80:
        return 3
    elif 60 <= n < 70:
        return 2
    else:
        return 1
for i in range(len(p1)):
    p1[i] = point_change(p1[i])
for i in range(len(p2)):
    p2[i] = point_change(p2[i])
for i in range(len(p3)):
    p3[i] = point_change(p3[i])

In [472]:
(sum(p1)*1 + sum(p2)*2 + sum(p3)*3) / (len(p1)*1 + len(p2)*2 + len(p3)*3)

3.7169811320754715